In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from dotenv import load_dotenv

load_dotenv('./env/.env')

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1
)

# k는 최대 저장할 메모리의 갯수 
memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({ "input:" : input }, { "output:" : output })

def get_history():
    return memory.load_memory_variables({})




add_message("Hi my name is michael, I live in seoul", "Hello Michael, nice to meet you")



In [6]:
add_message("I am a software engineer", "That's cool, what kind of software do you work on?")

In [ ]:
get_history()